NO LONGER VALID

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/nba_pred/'2019_2020'
%ls

/content/drive/My Drive/nba_pred/2019_2020
cur_ref_set.csv  TeamStats4.csv      TeamStatsMisc.csv  TeamStatsScore.csv
records.csv      TeamStatsAdvan.csv  TeamStatsOpp.csv   TeamStatsTrad.csv


In [ ]:
def getStats():
  import pandas as pd
  import numpy as np

  team_data_trad = pd.read_csv('TeamStatsTrad.csv')
  team_data_advan = pd.read_csv('TeamStatsAdvan.csv')
  team_data_misc = pd.read_csv('TeamStatsMisc.csv')
  team_data_4 = pd.read_csv('TeamStats4.csv')
  team_data_score = pd.read_csv('TeamStatsScore.csv')
  team_data_opp = pd.read_csv('TeamStatsOpp.csv')

  team_name = list(team_data_trad.columns)
  team_name.pop(0)

  team_stat = {}

  for name in team_name:
      team_stat[name] = list(team_data_trad[name])
      team_stat[name] += list(team_data_advan[name])
      team_stat[name] += list(team_data_misc[name])
      team_stat[name] += list(team_data_4[name])
      team_stat[name] += list(team_data_score[name])
      team_stat[name] += list(team_data_opp[name])
      team_stat[name] = [x for x in team_stat[name] if str(x) != 'nan']
  return team_stat

In [ ]:
def models(team_stat): 
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import normalize
  import pandas as pd
  import numpy as np

  records = pd.read_csv('records.csv')
  away_score = list(records['score_away'])
  home_score = list(records['score_home'])

  matrix1 = [] #data1 = home + away team stats + audience, target = home score;
  matrix2 = [] #data2 = away + home team stats + audience, target = away score
  for idx, row in records.iterrows():
    home_team = row['home']
    away_team = row['away']
    audience = row['audience']
    audience = audience.replace(',','')
    audience = int(audience)
    data1 = [*team_stat[home_team], *team_stat[away_team]]
    data2 = [*team_stat[away_team], *team_stat[home_team]]
    matrix1.append(data1)
    matrix2.append(data2)

  matrix1 = np.matrix(matrix1)
  matrix2 = np.matrix(matrix2)
  home_score = np.reshape(home_score, (len(home_score), 1))
  away_score = np.reshape(away_score, (len(away_score), 1))

  X_train1, X_test_home, y_train1, y_test_home = train_test_split(matrix1, home_score, test_size=0.3, shuffle=True)
  X_train2, X_test_away, y_train2, y_test_away = train_test_split(matrix2, away_score, test_size=0.3, shuffle=True)

  train_matrix = np.concatenate((X_train1, X_train2), axis=0)
  train_target = np.concatenate((y_train1, y_train2), axis=None)

  win_loss = []
  for home, away in zip(y_test_home, y_test_away):
    if (home > away):
      win_loss.append(1)
    else:
      win_loss.append(0)

  import tensorflow as tf
  from tensorflow.keras.layers import Dense
  from sklearn.metrics import accuracy_score
  from sklearn.linear_model import Ridge

  regression = Ridge()
  regression.fit(train_matrix, train_target)
  test_home = regression.predict(X_test_home)
  test_away = regression.predict(X_test_away)

  pred_win_loss = []
  for home, away in zip(test_home, test_away):
    if (home > away):
      pred_win_loss.append(1)
    else:
      pred_win_loss.append(0)

  #print('acc 1: ' + str(accuracy_score(win_loss, pred_win_loss)))

  model = tf.keras.Sequential()
  model.add(Dense(45, activation='relu'))
  model.add(Dense(22, activation='relu'))
  model.add(Dense(11, activation='relu'))
  model.add(Dense(1, activation='softplus'))
  model.compile(optimizer='adam', loss='mse')

  history = model.fit(train_matrix, train_target, verbose=0, batch_size=1)

  test_home = model.predict(X_test_home)
  test_away = model.predict(X_test_away)

  pred_win_loss = []
  for home, away in zip(test_home, test_away):
    if (home > away):
      pred_win_loss.append(1)
    else:
      pred_win_loss.append(0)

  #print('acc 2: ' + str(accuracy_score(win_loss, pred_win_loss)))
  return model, regression

In [ ]:
def predictionHelper(home_team, away_team, model, regression):
  import numpy as np
  home_stat = team_stat[home_team]
  away_stat = team_stat[away_team]

  data1 = [*home_stat, *away_stat]
  data2 = [*away_stat, *home_stat]

  matrix = []
  matrix.append(data1)
  matrix.append(data2)

  matrix = np.matrix(matrix)

  reg_pred = regression.predict(matrix)
  mod_pred = model.predict(matrix)
  mod_pred2 = model.predict(matrix)

  diff1 = float(reg_pred[0] - reg_pred[1])
  diff2 = float(mod_pred[0] - mod_pred[1])

  return diff1, diff2

In [ ]:
def countW(diff):
  win = 0
  for i in diff:
    if i > 0:
      win = win + 1
  
  return win

In [ ]:
def avg(diff):
  return float(sum(diff)/len(diff))

In [ ]:
def prediction(home_team, away_team, team_stat):
  reg_pred = []
  nn_pred = []
  for i in range(10):
    model, reg = models(team_stat)
    result1, result2 = predictionHelper(home_team, away_team, model, reg)
    reg_pred.append(result1)
    nn_pred.append(result2)
  
#  diff = []
#  for i in range(10):
#    diff.append(avg([reg_pred[i], nn_pred[i]]))

  print('out of 10 runs')
  print('Ridge prediction predicted {} wins for the '.format(countW(reg_pred)) + home_team + ', by avg of {:.2f} pts'.format(avg(reg_pred)))
  print('NN prediction predicted {} wins for the '.format(countW(nn_pred)) + home_team + ', by avg of {:.2f} pts'.format(avg(nn_pred)))
  #print('Overall prediction is {} wins for the '.format(countW(diff)) + home_team + ', by avg of {:.2f} pts'.format(avg(diff)))


In [ ]:
if __name__ == '__main__':
  team_stat = getStats()
  prediction('Dallas Mavericks', 'Los Angeles Lakers', team_stat)

out of 10 runs
Ridge prediction predicted 2 wins for the Dallas Mavericks, by avg of -1.84 pts
NN prediction predicted 5 wins for the Dallas Mavericks, by avg of -0.34 pts
